In [6]:
import pandas as pd
import os
import glob
import csv
import requests 
from pprint import pprint
import numpy as np

In [7]:

for i in range(16):
    path = os.path.join(r"../rawdata/iahhist_mf",f"'modified_df_{i}'")
    print(path)

../rawdata/iahhist_mf\'modified_df_0'
../rawdata/iahhist_mf\'modified_df_1'
../rawdata/iahhist_mf\'modified_df_2'
../rawdata/iahhist_mf\'modified_df_3'
../rawdata/iahhist_mf\'modified_df_4'
../rawdata/iahhist_mf\'modified_df_5'
../rawdata/iahhist_mf\'modified_df_6'
../rawdata/iahhist_mf\'modified_df_7'
../rawdata/iahhist_mf\'modified_df_8'
../rawdata/iahhist_mf\'modified_df_9'
../rawdata/iahhist_mf\'modified_df_10'
../rawdata/iahhist_mf\'modified_df_11'
../rawdata/iahhist_mf\'modified_df_12'
../rawdata/iahhist_mf\'modified_df_13'
../rawdata/iahhist_mf\'modified_df_14'
../rawdata/iahhist_mf\'modified_df_15'


In [8]:
# %%
# IMPORT MODULES
import pandas as pd
import os
import glob
import csv

# %%
# IMPORT FILE PATHS 
active_project = "../rawdata/iahhist_mf/"

file_paths = glob.glob(os.path.join(active_project, "*.csv"))

# %%
# setting up the list for all raw dfs
all_df = []

# listing turning movement counts as current 
current_csv_list = []

for file in file_paths:
    current_csv_list.append(file)

# %%
# function to read all csvs listed 

def read_csv(file_path):
    try:
        imp_df = pd.read_csv(file_path, 
                             sep='delimiter',
                             header=None,
                             engine='python'
                            )
        return imp_df #imp_df as in imported dataframe
    except pd.errors.ParserError as e:
        return (file_path, f"Error parsing CSV: {str(e)}")  # Return tuple with file path and error message
    except FileNotFoundError:
        return (file_path, "File not found")  # Return tuple with file path and error message

# %%
## validating dataframes - sometimes there's errors

for file_path in current_csv_list:
    imp_df = read_csv(file_path)  # Assign the returned DataFrame to imp_df
    all_df.append(imp_df)  # Append imp_df to all_df

valid_dfs = [df for df in all_df if isinstance(df, pd.DataFrame)]  # Filter out non-DataFrame objects

# %%
## modify each dataframe

def modify_in_df(df):
 # modifications to the df go here
    raw_data_startrow = df.index[df.iloc[:, 0].str.startswith("Carrier Code")].tolist()[0]
    raw_data = df.iloc[raw_data_startrow:, :]  # Extract from the starting row onwards
    raw_data.reset_index(drop=True, inplace=True)
    raw_data = raw_data.rename(columns={raw_data.columns[0]:"rawdata"})
    raw_table = raw_data["rawdata"].str.split(',',expand=True) #split columns
    raw_header = raw_table.iloc[0]
    raw_table.columns = raw_header  # Set the new header
    raw_table = raw_table.iloc[1:]  # Remove the first row
    #raw_table = raw_table.apply(lambda x: x.str.strip('"'))
    #raw_table["Date (MM/DD/YYYY)"] = pd.to_datetime(raw_table["Date (MM/DD/YYYY)"])
    #raw_table = raw_table[raw_table["Date (MM/DD/YYYY)"].dt.year > 2012]
    return raw_table 

# %%
## loop through dataframes

modified_df_list = []  # List to store the modified DataFrames
error_log = []  # List to store the error details
df = pd.DataFrame()

for i in range(len(valid_dfs)):
    try:
        df = valid_dfs[i]  # Retrieve the DataFrame by index
        modified_df = modify_in_df(df) # use this step to Modify the DataFrame
        modified_df_list.append(modified_df)  # Append the modified DataFrame to the list
        # Generate CSV file for the modified DataFrame
        # filename = f"modified_df_{i}.csv"
        #modified_df.to_csv(filename, index=False)
    except Exception as e:
        error_log.append((i, str(e)))  # Log the index and error message

# %%
big_df = pd.concat(modified_df_list, ignore_index=True)

historical_iah = big_df


In [9]:
#clean the data 
df = pd.DataFrame(historical_iah)
df_cleaned = df["Destination Airport"].unique()
airports = pd.DataFrame({"Airport Code": df_cleaned})

airports

,Airport Code
0,SLC
1,MSP
2,ATL
3,DTW
4,CVG
...,...
138,AGS
139,FCA
140,MDW
141,DRO


In [10]:
#set up the API 
api_key = input("what is your api key?")

In [15]:

#list for airport name,city and state 
airport_name =[]
airport_city = []
airport_state= []
lat = []
long = []

for x in range(143):
    airport = airports["Airport Code"][x]
    
    url = "https://airport-info.p.rapidapi.com/airport"
    querystring = {"iata": airport}
    headers = {
        "X-RapidAPI-Key": api_key,
		"X-RapidAPI-Host": "airport-info.p.rapidapi.com"
		}

    response = requests.get(url, headers=headers, params=querystring).json()
    # airport_name.append(response["name"])
    # airport_city.append(response["city"])
    # airport_state.append(response["state"])
    # lat.append(response["latitude"])
    # long.append(response["longitude"])
# airports

In [21]:
#add to the DF 

airports["Airport Name"]= airport_name
airports["City"]= airport_city
airports["State"] = airport_state
airports["Lat"] = lat
airports["Long"]=long
airports["Hotel Name"] = ""
airports["Hotel Address"] = ""
airports["Hotel Distance"] = ""
airports

airports.to_csv("CSV/Airportnames")

ValueError: Length of values (0) does not match length of index (143)

In [78]:
airports1 = pd.read_csv(r"CSV\Airportnames")
del airports1["Unnamed: 0"]
del airports1["Hotel Distance"]

In [79]:
airports1

,Airport Code,Airport Name,City,State,Lat,Long,Hotel Name,Hotel Address
0,MIA,Miami International Airport,Miami,Florida,25.795864,-80.287050,NaN,NaN
1,DFW,Dallas/Fort Worth International Airport,DFW Airport,Texas,32.899810,-97.040340,NaN,NaN
2,JFK,John F. Kennedy International Airport,NaN,New York,40.641310,-73.778140,NaN,NaN
3,CLT,Charlotte Douglas International Airport,Charlotte,North Carolina,35.214400,-80.947310,NaN,NaN
4,PHX,Phoenix Sky Harbor International Airport,Phoenix,Arizona,33.437267,-112.007790,NaN,NaN
...,...,...,...,...,...,...,...,...
138,SJT,San Angelo Regional Airport (Mathis Field),San Angelo,Texas,31.357288,-100.502940,NaN,NaN
139,JLN,Joplin Regional Airport,Webb City,Missouri,37.157475,-94.501595,NaN,NaN
140,DRO,Durango–La Plata County Airport,Durango,Colorado,37.150608,-107.753260,NaN,NaN
141,GSO,Piedmont Triad International Airport,Greensboro,North Carolina,36.104366,-79.935180,NaN,NaN


In [46]:
geoapify_key = input("what is your geoapify key?")


In [63]:
hotel_name = []
hotel_address = []

for x in range(143):
    latitude= airports1["Lat"][x]
    longitude = airports1["Long"][x]
    
    # Set the parameters for the type of search
    limit = 10
    radius = 2000
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"

    # set up a parameters dictionary
    params = {
        "categories": "accommodation.hotel",
        "filter":filters,
        "limit":limit,
        "bias":bias,
        "apiKey":geoapify_key
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    


    # Make an API request using the params dictionary
    hotel= requests.get(base_url, params=params).json()
    
    
    
    # Grab the first hotel from the results and store the details in the list
    try:
        hotel_name.append(hotel["features"][0]["properties"]["address_line1"])
        hotel_address.append(hotel["features"][0]["properties"]["address_line2"])
        # airports.loc[index, "distance"] = int(hotel["features"][0]["properties"]["distance"])
    except (KeyError, IndexError):
        # If no hotel or address is found append list with NAN
        hotel_name.append(np.nan)
        hotel_address.append(np.nan)



In [64]:
#check that we have 143 results 
print(len(hotel_name))
print(len(hotel_address))

143
143


In [80]:

airports1["Hotel Name"]= hotel_name
airports1["Hotel Address"]= hotel_address
airports1.to_csv("CSV/Hotels.csv")
airports1

,Airport Code,Airport Name,City,State,Lat,Long,Hotel Name,Hotel Address
0,MIA,Miami International Airport,Miami,Florida,25.795864,-80.287050,Miami International Airport Hotel,"2100 Northwest 42nd Avenue, Miami, FL 33142, U..."
1,DFW,Dallas/Fort Worth International Airport,DFW Airport,Texas,32.899810,-97.040340,Hyatt Regency DFW International Airport,"2334 Terminal C, Grapevine, TX 75261, United S..."
2,JFK,John F. Kennedy International Airport,NaN,New York,40.641310,-73.778140,TWA Hotel,"1 Idlewild Drive, New York, NY 11430, United S..."
3,CLT,Charlotte Douglas International Airport,Charlotte,North Carolina,35.214400,-80.947310,NaN,NaN
4,PHX,Phoenix Sky Harbor International Airport,Phoenix,Arizona,33.437267,-112.007790,SureStay Hotel by Best Western,"3037 East Van Buren Street, Phoenix, AZ 85008,..."
...,...,...,...,...,...,...,...,...
138,SJT,San Angelo Regional Airport (Mathis Field),San Angelo,Texas,31.357288,-100.502940,NaN,NaN
139,JLN,Joplin Regional Airport,Webb City,Missouri,37.157475,-94.501595,NaN,NaN
140,DRO,Durango–La Plata County Airport,Durango,Colorado,37.150608,-107.753260,NaN,NaN
141,GSO,Piedmont Triad International Airport,Greensboro,North Carolina,36.104366,-79.935180,Greensboro-High Point Marriott Airport,"1 Hotel Drive, Greensboro, NC 27409, United St..."


In [40]:
# geoapify_key = input("what is your geoapify key?")

# latitude= airports1["Lat"][2]
# longitude = airports1["Long"][2]
#     # Set the parameters for the type of search
# limit = 10
# radius = 2000
# filters = f"circle:{longitude},{latitude},{radius}"
# bias = f"proximity:{longitude},{latitude}"

# # set up a parameters dictionary
# params = {
#     "categories": "accommodation.hotel",
#     "filter":filters,
#     "limit":limit,
#     "bias":bias,
#     "apiKey":geoapify_key

#     }

#     # Set base URL
# base_url = "https://api.geoapify.com/v2/places"
# restaurant = requests.get(base_url, params=params)
    
#     # Convert the API response to JSON format
# restaurant = restaurant.json()

In [29]:
# import requests

# url = "https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-95.3676974,29.7589382,2000&bias=proximity:-95.3676974,29.7589382&limit=20&apiKey=d6039721557842c092a84b24a4eaf60b"
          
# response = requests.get(url)
# pprint(response.json())

{'features': [{'geometry': {'coordinates': [-95.36936055568076,
                                            29.756991962578375],
                            'type': 'Point'},
               'properties': {'address_line1': 'Hyatt Regency Houston',
                              'address_line2': '1200 Louisiana Street, '
                                               'Houston, TX 77002, United '
                                               'States of America',
                              'categories': ['accommodation',
                                             'accommodation.hotel',
                                             'building',
                                             'building.accommodation',
                                             'internet_access',
                                             'wheelchair',
                                             'wheelchair.yes'],
                              'city': 'Houston',
                              'country': '

In [ ]:

# #list for airport name,city and state 
# airport_name =[]
# airport_city = []
# airport_state= []
# lat = []
# long = []

# for x in range(143):
#     airport = airports["Airport Code"][x]
    
#     url = "https://airport-info.p.rapidapi.com/airport"
#     querystring = {"iata": airport}
#     headers = {
#         "X-RapidAPI-Key": api_key,
# 		"X-RapidAPI-Host": "airport-info.p.rapidapi.com"
# 		}

#     response = requests.get(url, headers=headers, params=querystring).json()
#     airport_name.append(response["name"])
#     airport_city.append(response["city"])
#     airport_state.append(response["state"])
#     lat.append(response["latitude"])
#     long.append(response["longitude"])
# # airports